![title](Prospect_utility.png)

![title](Prospect_utility2.png)

In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, timedelta
from datetime import datetime
from tqdm import tqdm
from dateutil.relativedelta import relativedelta
import statsmodels.api as sm

In [2]:
df=pd.read_csv("eb889d9be7128497.csv")

In [3]:
df.head()

,PERMNO,date,PRC,RET,SHROUT
0,10000,19851231,NaN,NaN,NaN
1,10000,19860131,-4.3750,C,3680.0
2,10000,19860228,-3.2500,-0.257143,3680.0
3,10000,19860331,-4.4375,0.365385,3680.0
4,10000,19860430,-4.0000,-0.098592,3793.0


In [4]:
df['PRC']=abs(df['PRC'])

In [5]:
df.dropna(inplace=True)

In [6]:
df = df[df.RET != 'C']

In [7]:
df['PERMNO'].nunique()

32107

In [8]:
df['date'] = df['date'].astype(str)
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')

In [9]:
df.set_index(df.date, inplace=True)
df.drop('date', axis=1, inplace=True)

In [10]:
df=df.sort_index()

In [11]:
df['PU']= pd.Series()

/Users/marcelo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


In [12]:
df.dtypes

PERMNO      int64
PRC       float64
RET        object
SHROUT    float64
PU        float64
dtype: object

In [13]:
df['RET'] = pd.to_numeric(df.RET, errors='coerce')

In [14]:
def prosp_utility(ret):
    alpha=0.5
    lambd=2.5
    gamma=0.61
    delta=0.69
    if ret >=0:
        return (ret**alpha)
    else:
        return (-lambd*(-ret)**alpha)

In [15]:
def w_P_pos(P):
    gamma=0.61
    return float((P**gamma)/(((P**gamma)+((1-P)**gamma))**(1/gamma)))

    
def w_P_neg(P): 
    delta=0.69
    return float((P**delta)/(((P**delta)+((1-P)**delta))**(1/delta)))

In [16]:
def pu_mod(full_permno_df):
    
    permno_id = full_permno_df.PERMNO.unique()[-1]
    
    if len(full_permno_df) <= 60:
        return []
    
    results = []
    for jump in range (60,len(full_permno_df)):
        permno_df=full_permno_df[jump-60:jump]
        list_final=permno_df.RET.sort_values()

        m=len(permno_df.loc[permno_df['RET'] < 0])
        n=60-m
        pu=[]

        for i in range(len(list_final)):
            if i==0:
                P_0=(1/60)
                pu.append(prosp_utility(list_final[i])*w_P_neg(P_0))
            elif i==59:
                P_0=(1/60)    
                pu.append(prosp_utility(list_final[i])*w_P_pos(P_0))
            else:
                if list_final[i]<0:
                    P_1=(1/60)*(i+1)
                    P_2=(1/60)*(i)
                    Pi= w_P_neg(P_1) - w_P_neg(P_2)
                    pu.append(prosp_utility(list_final[i])*Pi)
                else:
                    P_1=(1/60)*(60-i)
                    P_2=(1/60)*(60-i-1)                   
                    Pi= w_P_pos(P_1) - w_P_pos(P_2)
                    pu.append(prosp_utility(list_final[i])*Pi)
        results.append((permno_df.index[-1], permno_id, sum(pu)))
    return results
    
#         df.loc[(df.index==permno_df.index[-1]) & (df.PERMNO == ele), ['PU']]=sum(pu)



# Comment out lines below to generate pickle

In [17]:
#grps = df.groupby('PERMNO')

In [18]:
#grps = [x[1] for x in grps]

In [19]:
# import multiprocessing as mp

# with mp.Pool(mp.cpu_count()-1) as pool:
#     pool_result = pool.map(pu_mod, grps)

In [20]:
# import pickle

# with open('pool_results.pickle', 'wb') as f:
#     pickle.dump(pool_result, f)

# Analysis

In [21]:
with open('pool_results.pickle', 'rb') as f:
    mynewlist = pickle.load(f)

In [22]:
df=pd.read_csv("eb889d9be7128497.csv")

In [23]:
df['PRC']=abs(df['PRC'])

In [24]:
df.dropna(inplace=True)

In [25]:
df = df[df.RET != 'C']

In [26]:
df['date'] = df['date'].astype(str)
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')

In [27]:
df.set_index(df.date, inplace=True)
df.drop('date', axis=1, inplace=True)

In [28]:
df=df.sort_index()

In [29]:
df['PU']= pd.Series()

/Users/marcelo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


In [30]:
df['RET'] = pd.to_numeric(df.RET, errors='coerce')

In [31]:
for i in range(len(mynewlist)):
    for j in range(len(mynewlist[i])):
        mynewlist[i][j]=(mynewlist[i][j][0].strftime('%Y-%m-%d'), mynewlist[i][j][1], mynewlist[i][j][2])
    

In [32]:
new_data = [y for x in mynewlist for y in x if len(x) > 0]

In [33]:
new_df = pd.DataFrame(new_data, columns=['time', 'permno', 'pu'])

In [34]:
len(new_df)

2402002

In [35]:
len(new_df.loc[new_df.pu < 0])

2398278

In [36]:
df['time'] = pd.to_datetime(df.index)
new_df['time'] = pd.to_datetime(new_df['time'])

In [37]:
df.columns

Index(['PERMNO', 'PRC', 'RET', 'SHROUT', 'PU', 'time'], dtype='object')

In [38]:
final_df = df.merge(new_df, left_on=['time', 'PERMNO'], right_on=['time', 'permno'], how='left')

In [39]:
np.mean(np.isnan(final_df.pu))

0.39219351408771286

In [40]:
final_df.drop(['PU','permno'],axis=1,inplace=True)

In [41]:
final_df=final_df.dropna()
final_df.head()

,PERMNO,PRC,RET,SHROUT,time,pu
92536,17777,32.250,0.092437,1221.0,1964-12-31,-0.169802
92539,15915,16.125,-0.029630,2763.0,1964-12-31,-0.191615
92540,14816,12.500,0.049167,1586.0,1964-12-31,-0.144576
92541,17822,29.000,-0.016807,1697.0,1964-12-31,-0.177619
92545,21282,13.750,-0.009009,1508.0,1964-12-31,-0.188159


In [42]:
final_df=final_df.sort_values(['time', 'pu'], ascending=[True, True])

In [43]:
final_df['RET']=round(final_df['RET'],4)
final_df['pu']=round(final_df['pu'],4)


In [44]:
new_array=np.array(final_df['time'].unique())

In [45]:
df_temp=final_df[final_df['time']==new_array[0]]
x=pd.qcut(df_temp['pu'],10,retbins=True)
array_bins=np.array(list(x[1]))
array_bins

array([-0.292  , -0.20414, -0.18338, -0.1657 , -0.15052, -0.1364 ,
       -0.12224, -0.10858, -0.09534, -0.079  , -0.0373 ])

In [46]:
list_portfolios=['P1','P2','P3','P4','P5','P6','P7','P8','P9','P10']
d={}
for name in list_portfolios:
    d[name] = pd.DataFrame()

In [47]:
j=0
for name in d.keys():
        d[name]=df_temp[(df_temp.pu > array_bins[j]) & (df_temp.pu <= array_bins[j+1])]
        j+=1

In [48]:
for ele in list_portfolios:
    d[ele]=pd.DataFrame(d[ele])

In [49]:
for i in tqdm(range(1,len(new_array)-1)):
    df_temp=final_df[final_df['time']==new_array[i]]
    x=pd.qcut(df_temp['pu'],10,retbins=True)
    array_bins=np.array(list(x[1]))
    portfolio={}
    for name in list_portfolios:
        portfolio[name] = pd.DataFrame()
        j=0
    for name in portfolio.keys():
        portfolio[name]=df_temp[(df_temp.pu > array_bins[j]) & (df_temp.pu <= array_bins[j+1])]
        d[name]=pd.concat([d[name],portfolio[name]])
        j+=1
        

100%|██████████| 640/640 [01:22<00:00,  4.67it/s]


# 1) 

In [50]:
def avg_mkt_cap(name):
    list_mkt_cap=[]
    for name in list_portfolios:
        d[name]= d[name][(d[name].SHROUT != 0 ) & (d[name].PRC != 0 )]
        list_mkt_cap.append(np.mean(np.log(d[name].PRC*d[name].SHROUT)))
    return list_mkt_cap

In [51]:
avg_mkt=avg_mkt_cap(list_portfolios)

In [52]:
avg_mkt

[9.944366191149944,
 10.781329659556597,
 11.325922582028861,
 11.774475554480663,
 12.156451694682763,
 12.505609341589125,
 12.743027945524355,
 12.908290297879544,
 12.993808645233539,
 12.831078459241446]

In [53]:
def avg_return_t_1(name):
    mean_t_1=[]   
    for name in list_portfolios:
        array=[]
        for i in tqdm(range(len(d[name])-1)):
            pu123=d[name].pu.iloc[i:i+1]
            permno123=d[name].PERMNO.iloc[i:i+1]
            df_p1=final_df[(final_df['PERMNO']==int(permno123))]
            df_p1=df_p1.reset_index(drop=True)
            index_p1=df_p1.index[df_p1['pu'] == float(pu123)].tolist()
            new_index_p1=index_p1[0]+1
            try:
                array.append(df_p1[(df_p1.index==new_index_p1)].RET.values[0])
            except:
                continue

        d[name] = d[name][:-1]
        mean_t_1.append(np.mean(array))
    return mean_t_1

# Comment out lines below to calculate avg_returns at t+1

In [54]:
# avg_returns=avg_return_t_1(list_portfolios)

In [55]:
# avg_returns

In [56]:
#avg_returns.to_csv("")

# Create dataset to append findings

In [57]:
df_summary=pd.read_csv("avg_returns.csv")

In [58]:
df_summary.rename(columns={'Unnamed: 0': 'Portfolio'},inplace=True)

In [59]:
df_summary['Average Log Market Cap']=avg_mkt

In [60]:
df_summary

,Portfolio,Average Return t+1,Average Log Market Cap
0,P1,0.021637,9.944366
1,P2,0.013365,10.781330
2,P3,0.012085,11.325923
3,P4,0.012615,11.774476
4,P5,0.012440,12.156452
5,P6,0.012823,12.505609
6,P7,0.012177,12.743028
7,P8,0.012493,12.908290
8,P9,0.011913,12.993809
9,P10,0.009998,12.831078


In [61]:
market_df=pd.read_csv("rm_rf.CSV",index_col=0)

In [62]:
market_df=market_df.loc['196412':'201804']

In [63]:
market_df.head()

,rm_rf,rf
196412,0.0003,0.0031
196501,0.0354,0.0028
196502,0.0044,0.0030
196503,-0.0134,0.0036
196504,0.0311,0.0031


In [64]:
def capm(name):
    d_portfolio=d[name].groupby('time').mean()
    d_portfolio['rm_rf']=market_df['rm_rf'].values
    d_portfolio['rf']=market_df['rf'].values
    d_portfolio['RET']=d_portfolio['RET']-d_portfolio['rf']
    Y=d_portfolio['RET']
    X=d_portfolio['rm_rf']
    X_standard = sm.add_constant(X)
    model = sm.OLS(Y,X_standard)
    results = model.fit()
    print(results.summary())
    return results.params

In [65]:
list_coeffs=[]
for ele in list_portfolios:
    list_coeffs.append(capm(ele))

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.440
Model:                            OLS   Adj. R-squared:                  0.439
Method:                 Least Squares   F-statistic:                     501.2
Date:                Tue, 05 May 2020   Prob (F-statistic):           2.16e-82
Time:                        11:48:36   Log-Likelihood:                 811.36
No. Observations:                 641   AIC:                            -1619.
Df Residuals:                     639   BIC:                            -1610.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0212      0.003     -7.790      0.0

In [66]:
list_coeffs=pd.DataFrame(list_coeffs)

In [67]:
list_coeffs.rename(columns={"const":"CAPM alpha","rm_rf":"beta"},inplace=True)

In [68]:
df_summary['CAPM alpha']=list_coeffs['CAPM alpha']
df_summary['beta']=list_coeffs['beta']


In [69]:
df_summary

,Portfolio,Average Return t+1,Average Log Market Cap,CAPM alpha,beta
0,P1,0.021637,9.944366,-0.021170,1.364473
1,P2,0.013365,10.781330,-0.002934,1.319218
2,P3,0.012085,11.325923,0.001299,1.249083
3,P4,0.012615,11.774476,0.003522,1.180136
4,P5,0.012440,12.156452,0.005528,1.117843
5,P6,0.012823,12.505609,0.006116,1.033924
6,P7,0.012177,12.743028,0.007025,0.959389
7,P8,0.012493,12.908290,0.008689,0.863380
8,P9,0.011913,12.993809,0.010216,0.745411
9,P10,0.009998,12.831078,0.014449,0.582562


● Average log-market capitalization increases as we move from low prospective utility portfolios to high prospective utility portfolios.

● Average returns follow the opposite direction, as they decrease as we move from low prospective utility portfolios to high prospective utility ones.

● CAPM alphas are greater for high prospective utility portfolios than for low prospective utility ones. This shows that high prospective utility portfolios have a higher margin of overperformance compared to the market.

● Betas are all positive and they are greater for low prospective utility portfolios compared to high prospective utility ones. This reveals that all of the portfolios move in the same direction of the market, but the lower the prospective utility of the overall portfolio, the more volatile it will be compared to the market as a whole.

# 2)

In [70]:
d_long_short_portfolio=d['P1'].groupby('time').mean().RET-d['P10'].groupby('time').mean().RET

In [71]:
d_long_short_portfolio=pd.DataFrame(d_long_short_portfolio)

In [72]:
d_long_short_portfolio['rm_rf']=market_df['rm_rf'].values

In [73]:
d_long_short_portfolio['rf']=market_df['rf'].values

In [74]:
d_long_short_portfolio['RET-rf']=d_long_short_portfolio['RET']-d_long_short_portfolio['rf'].values

In [75]:
d_long_short_portfolio.head()

,RET,rm_rf,rf,RET-rf
time,,,,
1964-12-31,-0.042961,0.0003,0.0031,-0.046061
1965-01-29,0.046913,0.0354,0.0028,0.044113
1965-02-26,-0.000735,0.0044,0.0030,-0.003735
1965-03-31,-0.008507,-0.0134,0.0036,-0.012107
1965-04-30,-0.001248,0.0311,0.0031,-0.004348


In [76]:
p1_excess_return=np.mean(d_long_short_portfolio['RET-rf'])

In [77]:
Y=d_long_short_portfolio['RET-rf']
X=d_long_short_portfolio['rm_rf']
X_standard = sm.add_constant(X)
model = sm.OLS(Y,X_standard)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 RET-rf   R-squared:                       0.210
Model:                            OLS   Adj. R-squared:                  0.209
Method:                 Least Squares   F-statistic:                     170.1
Date:                Tue, 05 May 2020   Prob (F-statistic):           1.22e-34
Time:                        11:48:37   Log-Likelihood:                 817.89
No. Observations:                 641   AIC:                            -1632.
Df Residuals:                     639   BIC:                            -1623.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0395      0.003    -14.690      0.0

In [78]:
sharpe_ratio = d_long_short_portfolio['RET'].mean() / d_long_short_portfolio['RET'].std()
sharpe_ratio=sharpe_ratio*np.sqrt(12)
print(sharpe_ratio)

-1.445001145256369


In [79]:
d = {'CAPM alpha': results.params[0], 'Sharpe Ratio': sharpe_ratio}

In [80]:
d_sharpes=pd.DataFrame.from_dict(d,orient='index').T

In [81]:
d_sharpes

,CAPM alpha,Sharpe Ratio
0,-0.039519,-1.445001


# 3)

In [82]:
with open('pool_results_Q3.pickle', 'rb') as f:
    mynewlist = pickle.load(f)

In [83]:
for i in range(len(mynewlist)):
    for j in range(len(mynewlist[i])):
        mynewlist[i][j]=(mynewlist[i][j][0].strftime('%Y-%m-%d'), mynewlist[i][j][1], mynewlist[i][j][2])
    

In [84]:
new_data = [y for x in mynewlist for y in x if len(x) > 0]

In [85]:
new_df = pd.DataFrame(new_data, columns=['time', 'permno', 'pu'])

In [86]:
len(new_df)

2402002

In [87]:
len(new_df.loc[new_df.pu < 0])

2353697

In [88]:
new_df['time'] = pd.to_datetime(new_df['time'])

In [89]:
final_df = df.merge(new_df, left_on=['time', 'PERMNO'], right_on=['time', 'permno'], how='left')

In [90]:
np.mean(np.isnan(final_df.pu))

0.39219351408771286

In [91]:
final_df.drop(['PU','permno'],axis=1,inplace=True)

In [92]:
final_df=final_df.dropna()

In [93]:
final_df=final_df.sort_values(['time', 'pu'], ascending=[True, True])

In [94]:
final_df['RET']=round(final_df['RET'],4)
final_df['pu']=round(final_df['pu'],4)


In [95]:
new_array=np.array(final_df['time'].unique())

In [96]:
df_temp=final_df[final_df['time']==new_array[0]]
x=pd.qcut(df_temp['pu'],10,retbins=True)
array_bins=np.array(list(x[1]))
array_bins

array([-0.3392 , -0.2137 , -0.18256, -0.15884, -0.13896, -0.1212 ,
       -0.10144, -0.083  , -0.06558, -0.04512,  0.0129 ])

In [97]:
list_portfolios=['P1','P2','P3','P4','P5','P6','P7','P8','P9','P10']
d={}
for name in list_portfolios:
    d[name] = pd.DataFrame()

In [98]:
j=0
for name in d.keys():
        d[name]=df_temp[(df_temp.pu > array_bins[j]) & (df_temp.pu <= array_bins[j+1])]
        j+=1

In [99]:
for ele in list_portfolios:
    d[ele]=pd.DataFrame(d[ele])

In [100]:
for i in tqdm(range(1,len(new_array)-1)):
    df_temp=final_df[final_df['time']==new_array[i]]
    x=pd.qcut(df_temp['pu'],10,retbins=True)
    array_bins=np.array(list(x[1]))
    portfolio={}
    for name in list_portfolios:
        portfolio[name] = pd.DataFrame()
        j=0
    for name in portfolio.keys():
        portfolio[name]=df_temp[(df_temp.pu > array_bins[j]) & (df_temp.pu <= array_bins[j+1])]
        d[name]=pd.concat([d[name],portfolio[name]])
        j+=1
        

100%|██████████| 640/640 [01:16<00:00,  5.07it/s]


In [101]:
d_long_short_portfolio=d['P1'].groupby('time').mean().RET-d['P10'].groupby('time').mean().RET

In [102]:
d_long_short_portfolio=pd.DataFrame(d_long_short_portfolio)

In [103]:
d_long_short_portfolio['rm_rf']=market_df['rm_rf'].values

In [104]:
d_long_short_portfolio['rf']=market_df['rf'].values

In [105]:
d_long_short_portfolio['RET-rf']=d_long_short_portfolio['RET']-d_long_short_portfolio['rf'].values

In [106]:
d_long_short_portfolio.head()

,RET,rm_rf,rf,RET-rf
time,,,,
1964-12-31,-0.040760,0.0003,0.0031,-0.043860
1965-01-29,0.054972,0.0354,0.0028,0.052172
1965-02-26,0.004858,0.0044,0.0030,0.001858
1965-03-31,0.000064,-0.0134,0.0036,-0.003536
1965-04-30,-0.008414,0.0311,0.0031,-0.011514


In [107]:
Y=d_long_short_portfolio['RET-rf']
X=d_long_short_portfolio['rm_rf']
X_standard = sm.add_constant(X)
model = sm.OLS(Y,X_standard)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 RET-rf   R-squared:                       0.139
Model:                            OLS   Adj. R-squared:                  0.137
Method:                 Least Squares   F-statistic:                     102.9
Date:                Tue, 05 May 2020   Prob (F-statistic):           1.61e-22
Time:                        11:50:38   Log-Likelihood:                 769.88
No. Observations:                 641   AIC:                            -1536.
Df Residuals:                     639   BIC:                            -1527.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0333      0.003    -11.478      0.0

In [108]:
sharpe_ratio = d_long_short_portfolio['RET'].mean() / d_long_short_portfolio['RET'].std()
sharpe_ratio=sharpe_ratio*np.sqrt(12)
print(sharpe_ratio)

-1.1523975272097984


In [109]:
d = {'CAPM alpha': results.params[0], 'Sharpe Ratio': sharpe_ratio}

In [110]:
d=pd.DataFrame.from_dict(d,orient='index').T

In [111]:
d_sharpes=pd.concat([d_sharpes,d])

In [112]:
d_sharpes

,CAPM alpha,Sharpe Ratio
0,-0.039519,-1.445001
0,-0.033279,-1.152398


In [113]:
p2_excess_return=np.mean(d_long_short_portfolio['RET-rf'])

# 4)

In [114]:
with open('pool_results_Q4.pickle', 'rb') as f:
    mynewlist = pickle.load(f)

In [115]:
for i in range(len(mynewlist)):
    for j in range(len(mynewlist[i])):
        mynewlist[i][j]=(mynewlist[i][j][0].strftime('%Y-%m-%d'), mynewlist[i][j][1], mynewlist[i][j][2])
    

In [116]:
new_data = [y for x in mynewlist for y in x if len(x) > 0]

In [117]:
new_df = pd.DataFrame(new_data, columns=['time', 'permno', 'pu'])

In [118]:
len(new_df)

2402002

In [119]:
len(new_df.loc[new_df.pu < 0])

2288274

In [120]:
new_df['time'] = pd.to_datetime(new_df['time'])

In [121]:
final_df = df.merge(new_df, left_on=['time', 'PERMNO'], right_on=['time', 'permno'], how='left')

In [122]:
np.mean(np.isnan(final_df.pu))

0.39219351408771286

In [123]:
final_df.drop(['PU','permno'],axis=1,inplace=True)

In [124]:
final_df=final_df.dropna()

In [125]:
final_df=final_df.sort_values(['time', 'pu'], ascending=[True, True])

In [126]:
final_df['RET']=round(final_df['RET'],4)
final_df['pu']=round(final_df['pu'],4)


In [127]:
new_array=np.array(final_df['time'].unique())

In [128]:
df_temp=final_df[final_df['time']==new_array[0]]
x=pd.qcut(df_temp['pu'],10,retbins=True)
array_bins=np.array(list(x[1]))
array_bins

array([-0.0988 , -0.05664, -0.04678, -0.04014, -0.03412, -0.0288 ,
       -0.02474, -0.0208 , -0.01702, -0.01146,  0.0311 ])

In [129]:
list_portfolios=['P1','P2','P3','P4','P5','P6','P7','P8','P9','P10']
d={}
for name in list_portfolios:
    d[name] = pd.DataFrame()

In [130]:
j=0
for name in d.keys():
        d[name]=df_temp[(df_temp.pu > array_bins[j]) & (df_temp.pu <= array_bins[j+1])]
        j+=1

In [131]:
for ele in list_portfolios:
    d[ele]=pd.DataFrame(d[ele])

In [132]:
for i in tqdm(range(1,len(new_array)-1)):
    df_temp=final_df[final_df['time']==new_array[i]]
    x=pd.qcut(df_temp['pu'],10,retbins=True)
    array_bins=np.array(list(x[1]))
    portfolio={}
    for name in list_portfolios:
        portfolio[name] = pd.DataFrame()
        j=0
    for name in portfolio.keys():
        portfolio[name]=df_temp[(df_temp.pu > array_bins[j]) & (df_temp.pu <= array_bins[j+1])]
        d[name]=pd.concat([d[name],portfolio[name]])
        j+=1
        

100%|██████████| 640/640 [01:18<00:00,  5.47it/s]


In [133]:
d_long_short_portfolio=d['P1'].groupby('time').mean().RET-d['P10'].groupby('time').mean().RET

In [134]:
d_long_short_portfolio=pd.DataFrame(d_long_short_portfolio)

In [135]:
d_long_short_portfolio['rm_rf']=market_df['rm_rf'].values

In [136]:
d_long_short_portfolio['rf']=market_df['rf'].values

In [137]:
d_long_short_portfolio['RET-rf']=d_long_short_portfolio['RET']-d_long_short_portfolio['rf'].values

In [138]:
d_long_short_portfolio.head()

,RET,rm_rf,rf,RET-rf
time,,,,
1964-12-31,-0.028869,0.0003,0.0031,-0.031969
1965-01-29,0.005318,0.0354,0.0028,0.002518
1965-02-26,0.007180,0.0044,0.0030,0.004180
1965-03-31,-0.026981,-0.0134,0.0036,-0.030581
1965-04-30,-0.002226,0.0311,0.0031,-0.005326


In [139]:
Y=d_long_short_portfolio['RET-rf']
X=d_long_short_portfolio['rm_rf']
X_standard = sm.add_constant(X)
model = sm.OLS(Y,X_standard)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 RET-rf   R-squared:                       0.146
Model:                            OLS   Adj. R-squared:                  0.144
Method:                 Least Squares   F-statistic:                     108.9
Date:                Tue, 05 May 2020   Prob (F-statistic):           1.21e-23
Time:                        11:52:42   Log-Likelihood:                 995.45
No. Observations:                 641   AIC:                            -1987.
Df Residuals:                     639   BIC:                            -1978.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0553      0.002    -27.122      0.0

In [140]:
sharpe_ratio = d_long_short_portfolio['RET'].mean() / d_long_short_portfolio['RET'].std()
sharpe_ratio=sharpe_ratio*np.sqrt(12)
print(sharpe_ratio)

-3.0768003587614277


In [141]:
d = {'CAPM alpha': results.params[0], 'Sharpe Ratio': sharpe_ratio}

In [142]:
d=pd.DataFrame.from_dict(d,orient='index').T

In [143]:
d_sharpes=pd.concat([d_sharpes,d])

In [144]:
new_col = ['1', '2', '3'] 
d_sharpes.insert(loc=0, column='Step', value=new_col)

In [145]:
d_sharpes

,Step,CAPM alpha,Sharpe Ratio
0,1,-0.039519,-1.445001
0,2,-0.033279,-1.152398
0,3,-0.055309,-3.076800


In [146]:
p3_excess_return=np.mean(d_long_short_portfolio['RET-rf'])

In [147]:
excess_returns=np.array([p1_excess_return,p2_excess_return,p3_excess_return])

In [148]:
d_sharpes['Excess Return']=excess_returns

In [149]:
d_sharpes

,Step,CAPM alpha,Sharpe Ratio,Excess Return
0,1,-0.039519,-1.445001,-0.035481
0,2,-0.033279,-1.152398,-0.029894
0,3,-0.055309,-3.076800,-0.052860


● Comparing first the strategy that goes long P1/short P10(step 2) and the strategy where the investor perceives probability correctly(step 3), we see that the CAPM alphas are almost the same(-0.0395 vs -0.0332), and as they are both negative, this tells us that both portfolios underperformed the market.

● Now comparing the strategy that goes long P1/short P10(question 2) and the strategy where the investor’s preferences only exhibit loss aversion(step 4), the CAPM alphas differ by 0.02, as for the latter strategy the CAPM alpha is -0.0553, which is worse than the CAPM alpha of the former strategy. Furthermore, the Sharpe ratio is twice as bad (-3.07 vs -1.44) for the loss aversion strategy.